In [ ]:
import torch,sys,os
print("Python: ",sys.version.split()[0])
print("PyTorch: ",torch.__version__)
print("CUDA: ",torch.cuda.is_available())

from google.colab import drive
drive.mount('/content/drive')

PROJECT_ROOT ="/content/drive/MyDrive/cyberpunk_analyser"
os.makedirs(PROJECT_ROOT,exist_ok=True)
os.makedirs(os.path.join(PROJECT_ROOT,"data/images"),exist_ok=True)
os.makedirs(os.path.join(PROJECT_ROOT,"indexs"),exist_ok=True)
print("Project root:",PROJECT_ROOT)

Python:  3.12.12
PyTorch:  2.9.0+cu126
CUDA:  True
Mounted at /content/drive
Project root: /content/drive/MyDrive/cyberpunk_analyser


In [ ]:
!pip install transformers sentence-transformers faiss-cpu streamlit==1.25.0 pillow pandas tqdm accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 82.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 MB 76.6 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 114.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 158.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 144.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 62.3 MB/s  0:00:00
  Created wheel for pillow: filename=pillow-9.5.0-cp312-cp312-linux_x86_64.whl size=1210315 sha256=04486865164ce33acad93ed4a2d88270945a07f62cc9486f4c2e26369842694c
  Stored in directory: /root/.cache/pip/wheels/ea/de/2e/75a6399e5d8cd3a55c13c8f0658d996d4ce4cff37389

In [ ]:
import os,textwrap
SRC_DIR= os.path.join("/content","cyberpunk_src")
os.makedirs(SRC_DIR,exist_ok=True)



extractors = textwrap.dedent("""\
    from transformers import BlipForConditionalGeneration, BlipProcessor, CLIPModel, CLIPProcessor
    from sentence_transformers import SentenceTransformer
    from PIL import Image
    import numpy as np
    import torch

    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    # BLIP captioner (lightweight base)
    blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(DEVICE)

    # CLIP for image embeddings
    clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(DEVICE)
    clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    # Sentence Transformer for text embeddings
    sbert = SentenceTransformer("all-MiniLM-L6-v2", device=DEVICE)

    def generate_caption_pil(img_pil, max_length=50):
        inputs = blip_processor(images=img_pil, return_tensors="pt").to(DEVICE)
        out = blip_model.generate(**inputs, max_new_tokens=max_length)
        caption = blip_processor.decode(out[0], skip_special_tokens=True)
        return caption

    def generate_caption(image_path, max_length=50):
        img = Image.open(image_path).convert('RGB')
        return generate_caption_pil(img, max_length)

    def image_embedding(image_path):
        img = Image.open(image_path).convert('RGB')
        inputs = clip_processor(images=img, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            emb = clip_model.get_image_features(**inputs)
        emb = emb.cpu().numpy()[0].astype('float32')
        norm = np.linalg.norm(emb) + 1e-10
        return emb / norm

    def text_embedding(text):
        emb = sbert.encode(text, convert_to_numpy=True)
        emb = emb.astype('float32')
        norm = (np.linalg.norm(emb) + 1e-10)
        return emb / norm
""")
open(os.path.join(SRC_DIR, "extractors.py"), "w").write(extractors)

indexer = textwrap.dedent("""\
    import faiss
    import numpy as np
    import pickle

    class FaissIndex:
        def __init__(self, dim):
            self.dim = dim
            self.index = faiss.IndexFlatIP(dim)  # inner product; use normalized vectors
            self.id_map = []

        def add(self, vectors, ids):
            # vectors: (n, dim) numpy float32
            self.index.add(vectors)
            self.id_map.extend(ids)

        def search(self, qvec, top_k=5):
            qvec = qvec.reshape(1, -1).astype('float32')
            scores, idxs = self.index.search(qvec, top_k)
            results = []
            for s, i in zip(scores[0], idxs[0]):
                if i == -1:
                    continue
                results.append((self.id_map[i], float(s)))
            return results

        def save(self, base_path):
            faiss.write_index(self.index, base_path + ".index")
            with open(base_path + ".meta.pkl", "wb") as f:
                pickle.dump(self.id_map, f)

        def load(self, base_path):
            self.index = faiss.read_index(base_path + ".index")
            with open(base_path + ".meta.pkl", "rb") as f:
                self.id_map = pickle.load(f)
""")
open(os.path.join(SRC_DIR, "indexer.py"), "w").write(indexer)


rag = textwrap.dedent("""\
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
    import torch

    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    model_name = "google/flan-t5-small"  # runs on Colab; small but useful
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    rag_model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(DEVICE)

    def generate_answer(retrieved_texts, user_question, max_length=128):
        context = "\\n".join(retrieved_texts) if retrieved_texts else ""
        prompt = f"Context: {context}\\n\\nQuestion: {user_question}\\nAnswer concisely:"
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(DEVICE)
        with torch.no_grad():
            out = rag_model.generate(**inputs, max_new_tokens=max_length)
        return tokenizer.decode(out[0], skip_special_tokens=True)
""")
open(os.path.join(SRC_DIR, "rag.py"), "w").write(rag)

print("Wrote source files to", SRC_DIR)


Wrote source files to /content/cyberpunk_src


In [ ]:
import os
import textwrap

SRC_DIR = os.path.join("/content", "cyberpunk_src")
os.makedirs(SRC_DIR, exist_ok=True)


rag_code = textwrap.dedent("""\
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
    import torch

    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    model_name = "google/flan-t5-small"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    rag_model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(DEVICE)

    def generate_answer(retrieved_texts, user_question, max_length=128):
        # 1. Filter the list to ensure we only have strings
        clean_texts = []
        if isinstance(retrieved_texts, list):
            for t in retrieved_texts:
                # Only append if it is a string and not empty
                if isinstance(t, str) and t.strip():
                    clean_texts.append(t)

        # 2. Handle case where no valid text was found
        if not clean_texts:
            context = ""
        else:
            context = "\\n".join(clean_texts)

        # 3. Generate the answer
        prompt = f"Context: {context}\\n\\nQuestion: {user_question}\\nAnswer concisely:"
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(DEVICE)
        with torch.no_grad():
            out = rag_model.generate(**inputs, max_new_tokens=max_length)
        return tokenizer.decode(out[0], skip_special_tokens=True)
""")

with open(os.path.join(SRC_DIR, "rag.py"), "w") as f:
    f.write(rag_code)

print("SUCCESS: rag.py rewritten with type-checking safety.")

SUCCESS: rag.py rewritten with type-checking safety.


In [ ]:
!head -n 5 /content/cyberpunk_src/rag.py


from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "google/flan-t5-small"


In [ ]:

import pandas as pd, os
PROJECT_ROOT = "/content/drive/MyDrive/cyberpunk_analyser"
meta_path = os.path.join(PROJECT_ROOT, "data_metadata_template.csv")
sample = pd.DataFrame([
    {"id":"img_001","filename":"img_001.jpg","caption":"city neon alley with a holographic sign","tags":"city,neon,holo,alley","description":"A rainy neon alley with a lone courier and holographic signboards."},
    {"id":"img_002","filename":"img_002.jpg","caption":"cybernetic arm holding a compact drone","tags":"cyber-arm,drone,gadget","description":"Close-up of a cybernetic forearm holding a small reconnaissance drone."},
    {"id":"img_003","filename":"img_003.jpg","caption":"street vendor under neon umbrella","tags":"vendor,street,neon","description":"A street food vendor with bright neon lighting and holographic menu."},
    {"id":"img_004","filename":"img_004.jpg","caption":"armored figure with energy blade","tags":"armor,blade,weapon","description":"A combat-ready figure wearing reinforced armor and an energy blade on hip."},
    {"id":"img_005","filename":"img_005.jpg","caption":"holo-terminal and floating UI panels","tags":"ui,hologram,terminal","description":"Holographic terminal projecting floating user interface panels in a dim room."},
    {"id":"img_006","filename":"img_006.jpg"},
    {"id":"img_007","filename":"img_007.jpg"},

])
sample.to_csv(os.path.join(PROJECT_ROOT, "metadata.csv"), index=False)
print("Wrote metadata template to", os.path.join(PROJECT_ROOT, "metadata.csv"))
print("Now upload corresponding image files into:", os.path.join(PROJECT_ROOT, "data","images"))


Wrote metadata template to /content/drive/MyDrive/cyberpunk_analyser/metadata.csv
Now upload corresponding image files into: /content/drive/MyDrive/cyberpunk_analyser/data/images


In [ ]:


import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path


PROJECT_ROOT = "/content/drive/MyDrive/cyberpunk_analyser"
SRC_DIR = "/content/cyberpunk_src"

sys.path.append(SRC_DIR)


from extractors import image_embedding, text_embedding, generate_caption
from indexer import FaissIndex


meta_path = os.path.join(PROJECT_ROOT, "metadata.csv")
meta = pd.read_csv(meta_path)


image_vecs = []
image_ids  = []
text_vecs  = []
text_ids   = []


for idx, row in meta.iterrows():
    img_path = os.path.join(
    PROJECT_ROOT,
    "data_metadata_template.csv",
    row["filename"]
)




    if not os.path.exists(img_path):
        print("Missing", img_path, "- skipping")
        continue


    img_emb = image_embedding(img_path)
    image_vecs.append(img_emb)
    image_ids.append(row["id"])


    desc = row.get("description")
    cap  = row.get("caption")

    if pd.notna(desc) and isinstance(desc, str) and desc.strip():
        text = desc
    elif pd.notna(cap) and isinstance(cap, str) and cap.strip():
        text = cap
    else:

        print(f"Using BLIP for {row['id']}")
        text = generate_caption(img_path)


    txt_emb = text_embedding(text)
    text_vecs.append(txt_emb)
    text_ids.append(row["id"])


if len(image_vecs) > 0:
    image_vecs = np.vstack(image_vecs).astype("float32")
    text_vecs  = np.vstack(text_vecs).astype("float32")


    img_dim = image_vecs.shape[1]
    img_index = FaissIndex(img_dim)
    img_index.add(image_vecs, image_ids)
    img_index.save(os.path.join(PROJECT_ROOT, "indexes", "image_index"))


    txt_dim = text_vecs.shape[1]
    txt_index = FaissIndex(txt_dim)
    txt_index.add(text_vecs, text_ids)
    txt_index.save(os.path.join(PROJECT_ROOT, "indexes", "text_index"))

    print("Indexes built and saved to Drive.")

else:
    print("No images were indexed. Upload images and re-run.")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Using BLIP for img_006
Using BLIP for img_007
Indexes built and saved to Drive.


In [ ]:
import os, pandas as pd

PROJECT_ROOT = "/content/drive/MyDrive/cyberpunk_analyser"

print("metadata exists:", os.path.exists(os.path.join(PROJECT_ROOT, "metadata.csv")))
print("images:", os.listdir(os.path.join(PROJECT_ROOT, "data", "images")))
print("indexes:", os.listdir(os.path.join(PROJECT_ROOT, "indexes")))

df = pd.read_csv(os.path.join(PROJECT_ROOT, "metadata.csv"))
print("metadata rows:", len(df))
print(df[["id", "filename"]])


metadata exists: True
images: ['img_001.jpg', 'img_002.jpg', 'img_003.jpg', 'img_004.jpg', 'img_005.jpg', 'img_006.jpg', 'img_007.jpg']
indexes: ['image_index.index', 'image_index.meta.pkl', 'text_index.index', 'text_index.meta.pkl']
metadata rows: 7
        id     filename
0  img_001  img_001.jpg
1  img_002  img_002.jpg
2  img_003  img_003.jpg
3  img_004  img_004.jpg
4  img_005  img_005.jpg
5  img_006  img_006.jpg
6  img_007  img_007.jpg


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


In [ ]:
!ls /content/drive/MyDrive/cyberpunk_analyser/data_metadata_template


In [ ]:
ls /content/drive/MyDrive/cyberpunk_analyser


In [ ]:
INDEX_DIR = os.path.join(PROJECT_ROOT, "indexes")
os.makedirs(INDEX_DIR, exist_ok=True)

In [ ]:


from google.colab import files
import os
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

from extractors import generate_caption, image_embedding, text_embedding
from rag import generate_answer
from indexer import FaissIndex


PROJECT_ROOT = "/content/drive/MyDrive/cyberpunk_analyser"
INDEX_DIR = os.path.join(PROJECT_ROOT, "indexes")
META_PATH = os.path.join(PROJECT_ROOT, "metadata.csv")


uploaded = files.upload()

for name in uploaded:
    tmp_path = "/content/" + name
    print("\nUploaded:", name)


    caption = generate_caption(tmp_path)
    print("BLIP Caption:", caption)


    img_emb = image_embedding(tmp_path)

    img_idx = FaissIndex(img_emb.shape[0])
    img_idx.load(os.path.join(INDEX_DIR, "image_index"))

    sim_images = img_idx.search(img_emb, top_k=2)
    print("Top similar images:", sim_images)


    cap_emb = text_embedding(caption)

    text_idx = FaissIndex(cap_emb.shape[0])
    text_idx.load(os.path.join(INDEX_DIR, "text_index"))

    docs = text_idx.search(cap_emb, top_k=3)


    meta = pd.read_csv(META_PATH)

    retrieved_texts = []
    for did, score in docs:
        row = meta[meta["id"] == did]
        if not row.empty:
            desc = row.iloc[0].get("description")
            cap  = row.iloc[0].get("caption")
            if pd.notna(desc):
                retrieved_texts.append(desc)
            elif pd.notna(cap):
                retrieved_texts.append(cap)

    print("Retrieved docs:", retrieved_texts)


    question = input(
        "\nAsk a question about the image "
        "(e.g. 'show similar images', 'describe the scene'): "
    ).strip().lower()


    if "similar" in question and "image" in question:
        print("\nShowing similar images:\n")

        fig, axes = plt.subplots(1, len(sim_images), figsize=(15, 5))


        if len(sim_images) == 1:
            axes = [axes]

        for ax, (img_id, score) in zip(axes, sim_images):
            row = meta[meta["id"] == img_id]
            if row.empty:
                ax.axis("off")
                continue

            filename = row.iloc[0]["filename"]
            img_path = os.path.join(PROJECT_ROOT, "data_metadata_template.csv", filename)

            if os.path.exists(img_path):
                img = Image.open(img_path).convert("RGB")
                ax.imshow(img)
                ax.set_title(f"{img_id}\nscore={score:.2f}")
                ax.axis("off")
            else:
                ax.axis("off")

        plt.show()
        continue


    if question == "":
        question = (
            "Describe the scene and mention any notable objects or technology if visible. "
            "If nothing stands out, say so explicitly."
        )


    answer = generate_answer(retrieved_texts, question)

    print("\nQ:", question)
    print("A:", answer)


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
import os

path = "/content/drive/MyDrive/cyberpunk_analyser/data_metadata_template.csv"

print("Exists:", os.path.exists(path))
print("Is directory:", os.path.isdir(path))
print("Is file:", os.path.isfile(path))

print("\nContents:")
if os.path.isdir(path):
    print(os.listdir(path))


In [ ]:
!pip install -U pyngrok streamlit fastapi uvicorn nest-asyncio



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 126.2 MB/s  0:00:00
  Attempting uninstall: uvicorn
    Found existing installation: uvicorn 0.38.0
    Uninstalling uvicorn-0.38.0:
      Successfully uninstalled uvicorn-0.38.0
  Attempting uninstall: fastapi
    Found existing installation: fastapi 0.123.10
    Uninstalling fastapi-0.123.10:
      Successfully uninstalled fastapi-0.123.10
  Attempting uninstall: streamlit
    Found existing installation: streamlit 1.25.0
    Uninstalling streamlit-1.25.0:
      Successfully uninstalled streamlit-1.25.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [streamlit]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.21.0 requires fastapi<0.124.0,>=0.115.0, but you have fastapi 0.128.0 which is incompatible.
google-adk 1.21.0 requires tenacity<10.0.0,>=9.0.0, but you have tenacity 8.5.0 which is inco

In [ ]:
!rm -f /usr/local/bin/ngrok
!wget -q https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip
!unzip -o ngrok-v3-stable-linux-amd64.zip
!mv ngrok /usr/local/bin/ngrok
!chmod +x /usr/local/bin/ngrok
!ngrok version


In [ ]:
%%writefile /content/streamlit_app.py
import streamlit as st
import os, sys
import pandas as pd
from PIL import Image
import streamlit.components.v1 as components


st.set_page_config(
    layout="wide",
    page_title="CYBERPUNK"
)
st.markdown("""
<style>

/* ===== MATRIX RAIN OVERLAY (FIXED & BLENDED) ===== */
.matrix-layer {
    position: fixed;
    inset: 0;
    pointer-events: none;
    font-family: monospace;
    font-size: 12px;
    line-height: 14px;
    letter-spacing: 10px;
    white-space: pre-wrap;
    z-index: -10;
    text-shadow: 0 0 6px currentColor;
}

/* Cyan – main downward rain */
.matrix-cyan {
    color: rgba(0, 255, 255, 0.9);
    animation: matrixDown 9s linear infinite;
    opacity: 0.35;
}

/* Red – right side, upward */
.matrix-red {
    color: rgba(255, 40, 90, 0.85);
    animation: matrixUp 13s linear infinite;
    left: 60%;
    width: 40%;
    opacity: 0.35;
}

/* Yellow – slow depth layer */
.matrix-yellow {
    color: rgba(255, 220, 80, 0.45);
    animation: matrixDown 22s linear infinite;
    opacity: 0.25;
}

/* Green – LEFT blended rain */
.matrix-green-left {
    color: rgba(120, 255, 170, 0.55);
    animation: matrixDown 16s linear infinite;
    left: 0;
    width: 50%;
    opacity: 0.25;
}

/* Green – RIGHT blended rain */
.matrix-green-right {
    color: rgba(120, 255, 170, 0.45);
    animation: matrixUp 19s linear infinite;
    left: 50%;
    width: 50%;
    opacity: 0.22;
}

/* Animations */
@keyframes matrixDown {
    from { transform: translateY(-70%); }
    to   { transform: translateY(70%); }
}

@keyframes matrixUp {
    from { transform: translateY(70%); }
    to   { transform: translateY(-70%); }
}

/* Streamlit UI above background */
[data-testid="stAppViewContainer"] {
    position: relative;
    z-index: 10;
}

.stApp {
    background: #05070b;
}

[data-testid="stHeader"],
[data-testid="stToolbar"] {
    background: transparent !important;
}

</style>

<div class="matrix-layer matrix-cyan">
010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101
010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101
</div>

<div class="matrix-layer matrix-red">
101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010
101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010
</div>

<div class="matrix-layer matrix-yellow">
010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101
</div>

<div class="matrix-layer matrix-green-left">
010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101
010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101
</div>

<div class="matrix-layer matrix-green-right">
101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010
101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010
</div>
""", unsafe_allow_html=True)



st.markdown("""
<style>
html, body, [class*="css"] {
    background-color: #0b0e14;
    color: #e6e6e6;
}

.block-container {
    padding-top: 2rem;
    padding-left: 3rem;
    padding-right: 3rem;
}

button[kind="primary"] {
    background: linear-gradient(90deg,#00f0ff,#ff0055);
    border: none;
    color: black;
    font-weight: bold;
}
</style>
""", unsafe_allow_html=True)

st.markdown("""
<style>
.stApp,
[data-testid="stAppViewContainer"],
[data-testid="stHeader"],
[data-testid="stToolbar"] {
    background: transparent !important;
}

.block-container {
    padding-top: 2rem;
    padding-left: 3rem;
    padding-right: 3rem;
}

button[kind="primary"] {
    background: linear-gradient(90deg,#00f0ff,#ff0055);
    border: none;
    color: black;
    font-weight: bold;
}
</style>
""", unsafe_allow_html=True)

st.markdown("""
<style>
@import url('https://fonts.googleapis.com/css2?family=Rajdhani:wght@700&display=swap');

.cyberpunk-logo-wrap {
    display: flex;
    justify-content: center;
    margin-bottom: 35px;
}

.cyberpunk-logo {
    font-family: 'Rajdhani', sans-serif;
    font-size: 78px;
    font-weight: 700;
    letter-spacing: 6px;
    color: #00f0ff;
    display: inline-flex;
}

.cyberpunk-logo span {
    position: relative;
    display: inline-block;
    animation: glitchMove 2.5s infinite ease-in-out;
}

.cyberpunk-logo span:nth-child(odd) { animation-delay: .15s; }
.cyberpunk-logo span:nth-child(even) { animation-delay: .05s; }

.cyberpunk-logo span::before,
.cyberpunk-logo span::after {
    content: attr(data-char);
    position: absolute;
    left: 0;
    top: 0;
    opacity: 0.8;
}

.cyberpunk-logo span::before {
    color: #ff0055;
    transform: translate(-2px, 0);
    clip-path: polygon(0 0,100% 0,100% 45%,0 45%);
}

.cyberpunk-logo span::after {
    color: #00f0ff;
    transform: translate(2px, 0);
    clip-path: polygon(0 55%,100% 55%,100% 100%,0 100%);
}

@keyframes glitchMove {
    0%   { transform: translateY(0); }
    20%  { transform: translateY(-1.5px); }
    40%  { transform: translateY(1px); }
    60%  { transform: translateY(-1px); }
    80%  { transform: translateY(1.5px); }
    100% { transform: translateY(0); }
}

.cyberpunk-subtitle {
    text-align: center;
    margin-top: 8px;
    font-size: 13px;
    letter-spacing: 4px;
    color: #9aa0aa;
}
</style>

<div class="cyberpunk-logo-wrap">
    <div class="cyberpunk-logo">
        <span data-char="C">C</span>
        <span data-char="Y">Y</span>
        <span data-char="B">B</span>
        <span data-char="E">E</span>
        <span data-char="R">R</span>
        <span data-char="P">P</span>
        <span data-char="U">U</span>
        <span data-char="N">N</span>
        <span data-char="K">K</span>
    </div>
</div>

<div class="cyberpunk-subtitle">
    IMAGE • RETRIEVAL • RAG
</div>
""", unsafe_allow_html=True)


def panel(title, subtitle=None):
    st.markdown(f"""
    <div style="
        border: 1px solid rgba(0,240,255,0.25);
        border-left: 4px solid #00f0ff;
        padding: 18px 22px;
        margin-bottom: 22px;
        background: linear-gradient(
            145deg,
            rgba(20,25,40,0.95),
            rgba(10,14,20,0.95)
        );
    ">
        <div style="
            font-size: 13px;
            letter-spacing: 2px;
            color: #00f0ff;
            margin-bottom: 6px;
        ">
            {title.upper()}
        </div>
        {"<div style='font-size:12px;color:#888;margin-bottom:10px;'>"+subtitle+"</div>" if subtitle else ""}
    """, unsafe_allow_html=True)

def panel_end():
    st.markdown("</div>", unsafe_allow_html=True)

SRC_DIR = "/content/cyberpunk_src"
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

from extractors import generate_caption, image_embedding, text_embedding
from rag import generate_answer
from indexer import FaissIndex

PROJECT_ROOT = "/content/drive/MyDrive/cyberpunk_analyser"
INDEX_DIR = os.path.join(PROJECT_ROOT, "indexes")
META_PATH = os.path.join(PROJECT_ROOT, "metadata.csv")
IMAGES_DIR = os.path.join(PROJECT_ROOT, "data", "images")


@st.cache_data(show_spinner=False)
def load_metadata():
    if os.path.exists(META_PATH):
        return pd.read_csv(META_PATH)
    return pd.DataFrame(columns=["id","filename","caption","tags","description"])


@st.cache_resource(show_spinner=False)
def load_indexes():
    img_index = FaissIndex(1)
    txt_index = FaissIndex(1)
    img_index.load(os.path.join(INDEX_DIR, "image_index"))
    txt_index.load(os.path.join(INDEX_DIR, "text_index"))
    return img_index, txt_index

meta = load_metadata()
img_idx, txt_idx = load_indexes()


st.sidebar.header("Input Image")

dataset_files = meta["filename"].tolist() if not meta.empty else []
choice = st.sidebar.selectbox(
    "Choose source",
    ["Upload image"] + dataset_files
)


uploaded = st.file_uploader(
    "Upload an image",
    type=["jpg", "jpeg", "png"]
)


img = None
image_path = None

if uploaded is not None:
    try:
        img = Image.open(uploaded)
        img.verify()
        img = Image.open(uploaded)
        image_path = "/content/uploaded_image.png"
        with open(image_path, "wb") as f:
            f.write(uploaded.getbuffer())
    except Exception:
        img = None
        image_path = None

elif choice != "Upload image":
    candidate_path = os.path.join(IMAGES_DIR, choice)
    if os.path.exists(candidate_path):
        try:
            img = Image.open(candidate_path)
            img.verify()
            img = Image.open(candidate_path)
            image_path = candidate_path
        except Exception:
            img = None
            image_path = None


if img is not None and image_path is not None:

    panel("Input Image")
    st.image(img, caption="Input image", width=420)
    panel_end()

    panel("Generated Caption", "BLIP visual understanding")
    caption = generate_caption(image_path)
    st.write(caption)
    panel_end()

    panel("Similar Images", "FAISS image retrieval")
    img_emb = image_embedding(image_path)
    sim = img_idx.search(img_emb, top_k=6)

    for sid, score in sim:
        row = meta[meta["id"] == sid]
        if row.empty:
            continue

        filename = row.iloc[0]["filename"]
        desc = row.iloc[0]["description"]
        img_file = os.path.join(IMAGES_DIR, filename)

        if pd.isna(desc) or str(desc).strip().lower() == "nan":
            try:
                desc = generate_caption(img_file)
            except Exception:
                desc = "Description unavailable."

        st.markdown("""
        <div style="
            margin-bottom: 18px;
            padding: 14px;
            background: rgba(255,255,255,0.03);
            border-left: 3px solid #00f0ff;
        ">
        """, unsafe_allow_html=True)

        st.image(img_file, width=420)
        st.markdown(
            f"<div style='font-size:12px;color:#9aa0aa;margin-top:6px;'>"
            f"<b>{sid}</b> | similarity {score:.2f}"
            f"</div>",
            unsafe_allow_html=True
        )
        st.markdown(f"<div style='font-size:13px;margin-top:6px;'>{desc}</div>",
                    unsafe_allow_html=True)

        st.markdown("</div>", unsafe_allow_html=True)

    panel_end()

    panel("Ask a Question", "RAG answer generation")
    question = st.text_input(
        "What technology or weapon is visible and how might it be used?"
    )

    if st.button("Generate Answer", type="primary"):
        answer = generate_answer([caption], question)
        st.markdown("### Answer")
        st.write(answer)
    panel_end()

else:
    panel("Waiting for Input")
    st.info("Upload an image or select one from the dataset.")
    panel_end()


Overwriting /content/streamlit_app.py


In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("374oTb9lPDKZwvo39ovaUbWhgDR_872rZiByL7SCMkBxSTrDc")


In [ ]:
%%writefile /content/streamlit_app.py


In [ ]:
from pyngrok import ngrok
import nest_asyncio, subprocess, time

nest_asyncio.apply()


ngrok.set_auth_token("374oTb9lPDKZwvo39ovaUbWhgDR_872rZiByL7SCMkBxSTrDc")


ngrok.kill()


cmd = [
    "streamlit", "run", "/content/streamlit_app.py",
    "--server.port", "8501",
    "--server.headless", "true",
    "--server.enableCORS", "false"
]
subprocess.Popen(cmd)


time.sleep(10)


public_url = ngrok.connect(8501, bind_tls=True)
print("Streamlit public URL:", public_url)


Streamlit public URL: NgrokTunnel: "https://unsarcastical-amiya-crouchingly.ngrok-free.dev" -> "http://localhost:8501"
